In [93]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [94]:
data = pd.read_csv('Fifa21_train.csv')

In [95]:
def preprocess (data):
    data.columns = data.columns.str.lower()
    data['position'] = data['position'].fillna('CB')
    data = data.dropna()
    data = data.drop(['loan date end','bp','id','name','nationality','contract','club','team & contract','release clause','growth','joined','position','foot'], axis=1)
    data['height'] = data['height'].str.replace("'",".").str.replace("''","")
    data['height'] = data['height'].str.replace('"',"")
    data['weight'] = data['weight'].str.rstrip("lbs").astype(float)
    data['height'] = data['height'].astype(float)
    data['value'] = data['value'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')
    data['value'] = pd.to_numeric(data['value'])
    data['wage'] = data['wage'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')
    data['wage'] = pd.to_numeric(data['wage'])
    data['ls'] = data['ls'].str.split('+').str[0]
    
    return data 
    
modified_data = preprocess(data)
    

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17964\4252634381.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['value'] = data['value'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17964\4252634381.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['wage'] = data['wage'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.', '')


In [96]:
modified_data

,age,height,weight,value,wage,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
1,30,6.00,159.0,85000000,23000,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
37,20,6.20,176.0,450000,2000,267,27,64,62,57,57.0,220,54,40.0,31,34,61,246,53,54,37.0,60,42.0,312,63,70.0,56,69,54,259,59,30.0,62.0,48.0,60,52.0,70,24,25,21.0,57,12,10,9,12,14,1431,306,4 ★,2★,Medium,Medium,1 ★,54,61,44,54,29,64,7,61,61+2,61+2,54+0,58+0,58+0,58+0,54+0,55+2,55+2,55+2,53+2,51+2,51+2,51+2,53+2,41+2,43+2,43+2,43+2,41+2,40+2,42+2,42+2,42+2,40+2,17+2,61
58,30,6.00,163.0,600000,3000,299,58,59,64,62,56.0,299,65,55.0,56,60,63,359,79,73,75.0,64,68.0,339,65,74.0,73,71,56,313,67,64.0,63.0,65.0,54,66.0,192,65,66,61.0,58,8,16,12,8,14,1859,398,3 ★,3★,Medium,High,1 ★,76,59,61,66,65,71,7,64,64+2,64+2,64+0,64+0,64+0,64+0,64+0,64+2,64+2,64+2,65+2,63+2,63+2,63+2,65+2,65+2,64+2,64+2,64+2,65+2,65+2,65+2,65+2,65+2,65+2,17+2,67
66,31,6.00,176.0,27000000,15000,267,54,34,68,79,32.0,289,62,35.0,50,76,66,265,42,49,52.0,73,49.0,355,70,65.0,76,77,67,304,78,68.0,59.0,59.0,40,55.0,209,66,75,68.0,57,11,14,9,14,9,1746,369,3 ★,2★,Low,Medium,1 ★,46,49,66,62,70,76,6,59,59+1,59+1,59+0,60+0,60+0,60+0,59+0,63+1,63+1,63+1,61+1,69+1,69+1,69+1,61+1,66+1,72+1,72+1,72+1,66+1,66+1,71+1,71+1,71+1,66+1,19+1,73
80,21,6.30,176.0,425000,550,187,39,24,56,39,29.0,175,34,29.0,36,32,44,272,59,60,45.0,52,56.0,274,43,67.0,61,73,30,241,64,62.0,33.0,37.0,45,54.0,184,56,65,63.0,57,14,8,13,13,9,1390,297,3 ★,2★,Medium,Medium,1 ★,60,31,37,40,61,68,7,42,42+2,42+2,40+0,40+0,40+0,40+0,40+0,39+2,39+2,39+2,42+2,42+2,42+2,42+2,42+2,53+2,53+2,53+2,53+2,53+2,56+2,60+2,60+2,60+2,56+2,16+2,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11635,24,6.00,170.0,10000000,22000,334,77,64,66,72,55.0,357,82,77.0,54,65,79,371,76,76,78.0,72,69.0,344,75,68.0,67,67,67,321,51,56.0,74.0,73.0,67,75.0,166,57,57,52.0,64,11,15,11,15,12,1957,416,2 ★,4★,High,High,1 ★,76,67,72,80,57,64,81,72,72+2,72+2,75+0,74+0,74+0,74+0,75+0,74+2,74+2,74+2,75+2,71+2,71+2,71+2,75+2,67+2,64+2,64+2,64+2,67+2,65+2,61+2,61+2,61+2,65+2,20+2,76
11641,27,6.00,176.0,9000000,60000,378,69,79,79,75,76.0,310,71,65.0,57,45,72,374,78,79,75.0,73,69.0,382,78,88.0,77,72,67,296,66,18.0,79.0,63.0,70,78.0,83,32,28,23.0,51,12,14,9,7,9,1874,398,4 ★,3★,High,High,1 ★,79,76,66,72,32,73,236,76,76+1,76+1,73+0,74+0,74+0,74+0,73+0,71+2,71+2,71+2,72+2,64+2,64+2,64+2,72+2,55+2,51+2,51+2,51+2,55+2,52+2,48+2,48+2,48+2,52+2,17+2,76
11669,20,6.10,170.0,550000,2000,265,42,62,51,53,57.0,257,62,56.0,37,41,61,327,70,74,59.0,58,66.0,306,70,56.0,55,65,60,239,40,16.0,54.0,60.0,69,56.0,66,28,21,17.0,42,7,8,8,11,8,1502,328,3 ★,2★,Medium,Medium,1 ★,72,63,50,61,25,57,29,61,61+2,61+2,59+0,61+0,61+0,61+0,59+0,59+2,59+2,59+2,58+2,52+2,52+2,52+2,58+2,43+2,40+2,40+2,40+2,43+2,40+2,37+2,37+2,37+2,40+2,14+2,61
11691,29,5.80,168.0,26000000,12000,309,77,47,57,71,57.0,354,59,82.0,72,72,69,348,

In [97]:
y = modified_data['ova'] #target variable
X = modified_data.drop(['ova'], axis=1) #dropped target variable
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [98]:
X_cat

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
1,3 ★,4★,High,Low,2 ★,44,77,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
37,4 ★,2★,Medium,Medium,1 ★,7,61,61+2,61+2,54+0,58+0,58+0,58+0,54+0,55+2,55+2,55+2,53+2,51+2,51+2,51+2,53+2,41+2,43+2,43+2,43+2,41+2,40+2,42+2,42+2,42+2,40+2,17+2
58,3 ★,3★,Medium,High,1 ★,7,64,64+2,64+2,64+0,64+0,64+0,64+0,64+0,64+2,64+2,64+2,65+2,63+2,63+2,63+2,65+2,65+2,64+2,64+2,64+2,65+2,65+2,65+2,65+2,65+2,65+2,17+2
66,3 ★,2★,Low,Medium,1 ★,6,59,59+1,59+1,59+0,60+0,60+0,60+0,59+0,63+1,63+1,63+1,61+1,69+1,69+1,69+1,61+1,66+1,72+1,72+1,72+1,66+1,66+1,71+1,71+1,71+1,66+1,19+1
80,3 ★,2★,Medium,Medium,1 ★,7,42,42+2,42+2,40+0,40+0,40+0,40+0,40+0,39+2,39+2,39+2,42+2,42+2,42+2,42+2,42+2,53+2,53+2,53+2,53+2,53+2,56+2,60+2,60+2,60+2,56+2,16+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11635,2 ★,4★,High,High,1 ★,81,72,72+2,72+2,75+0,74+0,74+0,74+0,75+0,74+2,74+2,74+2,75+2,71+2,71+2,71+2,75+2,67+2,64+2,64+2,64+2,67+2,65+2,61+2,61+2,61+2,65+2,20+2
11641,4 ★,3★,High,High,1 ★,236,76,76+1,76+1,73+0,74+0,74+0,74+0,73+0,71+2,71+2,71+2,72+2,64+2,64+2,64+2,72+2,55+2,51+2,51+2,51+2,55+2,52+2,48+2,48+2,48+2,52+2,17+2
11669,3 ★,2★,Medium,Medium,1 ★,29,61,61+2,61+2,59+0,61+0,61+0,61+0,59+0,59+2,59+2,59+2,58+2,52+2,52+2,52+2,58+2,43+2,40+2,40+2,40+2,43+2,40+2,37+2,37+2,37+2,40+2,14+2
11691,2 ★,3★,Medium,Medium,1 ★,3,62,62+2,62+2,65+0,64+0,64+0,64+0,65+0,66+2,66+2,66+2,67+2,68+2,68+2,68+2,67+2,70+2,69+2,69+2,69+2,70+2,70+2,65+2,65+2,65+2,70+2,16+2


In [99]:
def preprocess2 (X_cat):
    for col, series in X_cat.iteritems():
        X_cat[col] = series.apply(lambda x: x.split('+')[0] if '+' in x else x)
    for col, series in X_cat.iteritems():
        X_cat[col] = series.apply(lambda x: x.replace('★', ' ') if '★' in x else x)
    X_cat = X_cat.apply(pd.to_numeric, errors = 'ignore')
    
    return X_cat

modified_data2 = preprocess2(X_cat)
dataframe = pd.concat([X_cat, X_num], axis=1)

    


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17964\3338076215.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in X_cat.iteritems():
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17964\3338076215.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in X_cat.iteritems():


In [100]:
dataframe.dtypes

w/f    object
sm     object
a/w    object
d/w    object
ir     object
        ...  
sho     int64
pas     int64
dri     int64
def     int64
phy     int64
Length: 87, dtype: object

In [101]:
dataframe

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,height,weight,value,wage,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy
1,3,4,High,Low,2,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,30,6.00,159.0,85000000,23000,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71
37,4,2,Medium,Medium,1,7,61,61,61,54,58,58,58,54,55,55,55,53,51,51,51,53,41,43,43,43,41,40,42,42,42,40,17,20,6.20,176.0,450000,2000,267,27,64,62,57,57.0,220,54,40.0,31,34,61,246,53,54,37.0,60,42.0,312,63,70.0,56,69,54,259,59,30.0,62.0,48.0,60,52.0,70,24,25,21.0,57,12,10,9,12,14,1431,306,54,61,44,54,29,64
58,3,3,Medium,High,1,7,64,64,64,64,64,64,64,64,64,64,64,65,63,63,63,65,65,64,64,64,65,65,65,65,65,65,17,30,6.00,163.0,600000,3000,299,58,59,64,62,56.0,299,65,55.0,56,60,63,359,79,73,75.0,64,68.0,339,65,74.0,73,71,56,313,67,64.0,63.0,65.0,54,66.0,192,65,66,61.0,58,8,16,12,8,14,1859,398,76,59,61,66,65,71
66,3,2,Low,Medium,1,6,59,59,59,59,60,60,60,59,63,63,63,61,69,69,69,61,66,72,72,72,66,66,71,71,71,66,19,31,6.00,176.0,27000000,15000,267,54,34,68,79,32.0,289,62,35.0,50,76,66,265,42,49,52.0,73,49.0,355,70,65.0,76,77,67,304,78,68.0,59.0,59.0,40,55.0,209,66,75,68.0,57,11,14,9,14,9,1746,369,46,49,66,62,70,76
80,3,2,Medium,Medium,1,7,42,42,42,40,40,40,40,40,39,39,39,42,42,42,42,42,53,53,53,53,53,56,60,60,60,56,16,21,6.30,176.0,425000,550,187,39,24,56,39,29.0,175,34,29.0,36,32,44,272,59,60,45.0,52,56.0,274,43,67.0,61,73,30,241,64,62.0,33.0,37.0,45,54.0,184,56,65,63.0,57,14,8,13,13,9,1390,297,60,31,37,40,61,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11635,2,4,High,High,1,81,72,72,72,75,74,74,74,75,74,74,74,75,71,71,71,75,67,64,64,64,67,65,61,61,61,65,20,24,6.00,170.0,10000000,22000,334,77,64,66,72,55.0,357,82,77.0,54,65,79,371,76,76,78.0,72,69.0,344,75,68.0,67,67,67,321,51,56.0,74.0,73.0,67,75.0,166,57,57,52.0,64,11,15,11,15,12,1957,416,76,67,72,80,57,64
11641,4,3,High,High,1,236,76,76,76,73,74,74,74,73,71,71,71,72,64,64,64,72,55,51,51,51,55,52,48,48,48,52,17,27,6.00,176.0,9000000,60000,378,69,79,79,75,76.0,310,71,65.0,57,45,72,374,78,79,75.0,73,69.0,382,78,88.0,77,72,67,296,66,18.0,79.0,63.0,70,78.0,83,32,28,23.0,51,12,14,9,7,9,1874,398,79,76,66,72,32,73
11669,3,2,Medium,Medium,1,29,61,61,61,59,61,61,61,59,59,59,59,58,52,52,52,58,43,40,40,40,43,40,37,37,37,40,14,20,6.10,170.0,550000,2000,265,42,62,51,53,57.0,257,62,56.0,37,41,61,327,70,74,59.0,58,66.0,306,70,56.0,55,65,60,239,40,16.0,54.0,60.0,69,56.0,66,28,21,17.0,42,7,8,8,11,8,1502,328,72,63,50,61,25,57
11691,2,3,Medium,Medium,1,3,62,62,62,65,64,64,64,65,66,66,66,67,68,68,68,67,70,69,69,69,70,70,65,65,65,70,16,29,5.80,168.0,26000000,12000,309,77,47,57,71,57.0,354,59,82.0,72,72,69,348,77,75,73.0,56,67.0,359,80,58.0,86,68,67,311,59,66.0,57.0,72.0,57,52.0,204,67,69,68.0,55,15,11,13,8,8,1940,409,76,59,73,64,67,70


In [102]:
pd.set_option('display.max_columns', None)
dataframe

,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,height,weight,value,wage,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy
1,3,4,High,Low,2,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,30,6.00,159.0,85000000,23000,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71
37,4,2,Medium,Medium,1,7,61,61,61,54,58,58,58,54,55,55,55,53,51,51,51,53,41,43,43,43,41,40,42,42,42,40,17,20,6.20,176.0,450000,2000,267,27,64,62,57,57.0,220,54,40.0,31,34,61,246,53,54,37.0,60,42.0,312,63,70.0,56,69,54,259,59,30.0,62.0,48.0,60,52.0,70,24,25,21.0,57,12,10,9,12,14,1431,306,54,61,44,54,29,64
58,3,3,Medium,High,1,7,64,64,64,64,64,64,64,64,64,64,64,65,63,63,63,65,65,64,64,64,65,65,65,65,65,65,17,30,6.00,163.0,600000,3000,299,58,59,64,62,56.0,299,65,55.0,56,60,63,359,79,73,75.0,64,68.0,339,65,74.0,73,71,56,313,67,64.0,63.0,65.0,54,66.0,192,65,66,61.0,58,8,16,12,8,14,1859,398,76,59,61,66,65,71
66,3,2,Low,Medium,1,6,59,59,59,59,60,60,60,59,63,63,63,61,69,69,69,61,66,72,72,72,66,66,71,71,71,66,19,31,6.00,176.0,27000000,15000,267,54,34,68,79,32.0,289,62,35.0,50,76,66,265,42,49,52.0,73,49.0,355,70,65.0,76,77,67,304,78,68.0,59.0,59.0,40,55.0,209,66,75,68.0,57,11,14,9,14,9,1746,369,46,49,66,62,70,76
80,3,2,Medium,Medium,1,7,42,42,42,40,40,40,40,40,39,39,39,42,42,42,42,42,53,53,53,53,53,56,60,60,60,56,16,21,6.30,176.0,425000,550,187,39,24,56,39,29.0,175,34,29.0,36,32,44,272,59,60,45.0,52,56.0,274,43,67.0,61,73,30,241,64,62.0,33.0,37.0,45,54.0,184,56,65,63.0,57,14,8,13,13,9,1390,297,60,31,37,40,61,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11635,2,4,High,High,1,81,72,72,72,75,74,74,74,75,74,74,74,75,71,71,71,75,67,64,64,64,67,65,61,61,61,65,20,24,6.00,170.0,10000000,22000,334,77,64,66,72,55.0,357,82,77.0,54,65,79,371,76,76,78.0,72,69.0,344,75,68.0,67,67,67,321,51,56.0,74.0,73.0,67,75.0,166,57,57,52.0,64,11,15,11,15,12,1957,416,76,67,72,80,57,64
11641,4,3,High,High,1,236,76,76,76,73,74,74,74,73,71,71,71,72,64,64,64,72,55,51,51,51,55,52,48,48,48,52,17,27,6.00,176.0,9000000,60000,378,69,79,79,75,76.0,310,71,65.0,57,45,72,374,78,79,75.0,73,69.0,382,78,88.0,77,72,67,296,66,18.0,79.0,63.0,70,78.0,83,32,28,23.0,51,12,14,9,7,9,1874,398,79,76,66,72,32,73
11669,3,2,Medium,Medium,1,29,61,61,61,59,61,61,61,59,59,59,59,58,52,52,52,58,43,40,40,40,43,40,37,37,37,40,14,20,6.10,170.0,550000,2000,265,42,62,51,53,57.0,257,62,56.0,37,41,61,327,70,74,59.0,58,66.0,306,70,56.0,55,65,60,239,40,16.0,54.0,60.0,69,56.0,66,28,21,17.0,42,7,8,8,11,8,1502,328,72,63,50,61,25,57
11691,2,3,Medium,Medium,1,3,62,62,62,65,64,64,64,65,66,66,66,67,68,68,68,67,70,69,69,69,70,70,65,65,65,70,16,29,5.80,168.0,26000000,12000,309,77,47,57,71,57.0,354,59,82.0,72,72,69,348,77,75,73.0,56,67.0,359,80,58.0,86,68,67,311,59,66.0,57.0,72.0,57,52.0,204,67,69,68.0,55,15,11,13,8,8,1940,409,76,59,73,64,67,70
